In [2]:
%pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/c1/cf/a662bc8f40588d54663edfe12980946670490bff0b6e793c7896a4fe36df/xgboost-2.0.0-py3-none-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 8.3 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [20]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


#  Modelado y evaluación
# -----------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRFRegressor
import xgboost as xgb

In [5]:
df = pd.read_pickle("../data/bikes_registrados_estand_encod_completo.pkl")
df.head()

,estacion,año,mes,festivo,dia,meteo,temperatura,humedad,velo_viento,usu_registrados,laborable_laborable,laborable_no laborable
0,0,0,0,0,0,1,-0.486274,0.858854,-0.208909,654,0,1
1,0,0,0,1,0,1,-0.425690,0.331548,0.687550,670,1,0
2,0,0,0,1,1,2,-0.949970,-0.912000,0.685210,1229,1,0
3,0,0,0,1,1,2,-0.938563,-0.176088,-0.210435,1454,1,0
4,0,0,0,1,1,2,-0.853992,-0.913519,0.060294,1518,1,0


In [6]:
X = df.drop("usu_registrados", axis = 1)
y = df["usu_registrados"]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)


In [10]:
# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

/home/arillien/anaconda3/envs/Proyecto3/lib/python3.10/site-packages/xgboost/data.py:463: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  or is_sparse(dtype)
/home/arillien/anaconda3/envs/Proyecto3/lib/python3.10/site-packages/xgboost/data.py:464: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  or (is_categorical_dtype(dtype) and enable_categorical)
/home/arillien/anaconda3/envs/Proyecto3/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/arillien/anaconda3/envs/Proyecto3/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) inste

In [11]:
params = {"objective": "reg:squarederror", "tree_method": "hist"}

In [12]:
n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
)

In [13]:
preds = model.predict(dtest_reg)

In [15]:
rmse = mean_squared_error(y_test, preds, squared=False)

print(f"RMSE of the base model: {rmse:.3f}")

RMSE of the base model: 674.030


In [16]:
y_pred_test_rf = model.predict(dtest_reg)
y_pred_train_rf = model.predict(dtrain_reg)

In [17]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [19]:
xgb_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "XGBoost")
xgb_results

,MAE,MSE,RMSE,R2,set,modelo
0,501.630798,454316.146902,674.029782,0.831310,test,XGBoost
1,19.647642,767.350365,27.701090,0.999675,train,XGBoost


<h2> RANDOM FOREST </h2>

In [41]:
m = len(X.columns)

params = dict(
    objective="reg:squarederror",
    learning_rate=1,
    num_parallel_tree=500,
    subsample=0.63,
    colsample_bynode=int(np.sqrt(m))/m,
    reg_lambda=0,
    max_depth=20,
    min_child_weight=2
)

In [42]:
rf_xgb = xgb.train(  # Wall time 34 s
    params, 
    dtrain_reg, 
    num_boost_round=1
)

In [45]:
y_pred_test_xgbrf = rf_xgb.predict(dtest_reg)
y_pred_train_xgbrf = rf_xgb.predict(dtrain_reg)

In [46]:
xgb_rf_results = metricas(y_test, y_train, y_pred_test_xgbrf, y_pred_train_xgbrf, "XGBoost RF")
xgb_rf_results

,MAE,MSE,RMSE,R2,set,modelo
0,499.296396,419601.538357,647.766577,0.844199,test,XGBoost RF
1,316.574102,188937.544694,434.669466,0.919914,train,XGBoost RF
